# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Define features and target
features = spaceship.drop(columns=['Transported'])
target = spaceship['Transported']

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Handle missing values
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train.select_dtypes(include=[np.number]))
X_test_imputed = imputer.transform(X_test.select_dtypes(include=[np.number]))

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the training features and transform them
X_train_scaled = scaler.fit_transform(X_train_imputed)

# Transform the test features using the fitted scaler
X_test_scaled = scaler.transform(X_test_imputed)

print("Features scaled using StandardScaler.")

Features scaled using StandardScaler.


**Perform Train Test Split**

In [7]:
# Perform one-hot encoding on categorical features
spaceship_dummies = pd.get_dummies(spaceship.drop(columns=['Transported']), drop_first=True)

# Separate features (X) and target (y)
X = spaceship_dummies
y = spaceship['Transported'].astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the training features and transform them
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test features using the fitted scaler
X_test_scaled = scaler.transform(X_test)

print("Train and test sets created and features scaled.")
print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

Train and test sets created and features scaled.
X_train_scaled shape: (6954, 23735)
X_test_scaled shape: (1739, 23735)
y_train shape: (6954,)
y_test shape: (1739,)


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [11]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Initialize the base estimator (KNN Classifier)
knn = KNeighborsClassifier(n_neighbors=5)

# Implement Bagging
bagging_clf = BaggingClassifier(estimator=knn,
                                n_estimators=5,  # Reduced number of estimators
                                random_state=42,
                                oob_score=True,
                                n_jobs=-1)

# Train the Bagging ensemble
bagging_clf.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred_bagging = bagging_clf.predict(X_test_scaled)

# Evaluate the Bagging ensemble
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
print("Bagging Classifier Accuracy:", accuracy_bagging)
print("Bagging Classifier OOB Score:", bagging_clf.oob_score_)

# Implement Pasting
pasting_clf = BaggingClassifier(estimator=knn,
                                n_estimators=5,  # Reduced number of estimators
                                random_state=42,
                                bootstrap=False,
                                n_jobs=-1)

# Train the Pasting ensemble
pasting_clf.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred_pasting = pasting_clf.predict(X_test_scaled)

# Evaluate the Pasting ensemble
accuracy_pasting = accuracy_score(y_test, y_pred_pasting)
print("Pasting Classifier Accuracy:", accuracy_pasting)

c:\Users\cleid\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_bagging.py:865: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
c:\Users\cleid\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_bagging.py:871: RuntimeWarning: invalid value encountered in divide
  oob_decision_function = predictions / predictions.sum(axis=1)[:, np.newaxis]


Bagging Classifier Accuracy: 0.4945370902817711
Bagging Classifier OOB Score: 0.4966925510497555
Pasting Classifier Accuracy: 0.49511213341000576


- Random Forests

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming X and y are your feature matrix and target vector
# and they have been split into X_train, X_test, y_train, y_test
# It's generally a good idea to work with unscaled data for tree-based models like Random Forests

# Initialize the Random Forest Classifier
# You can adjust hyperparameters like n_estimators, max_depth, etc.
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# Train the Random Forest model
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_classifier.predict(X_test)

# Evaluate the Random Forest model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

# Print the evaluation metrics
print("Random Forest Classifier Evaluation:")
print(f"Accuracy: {accuracy_rf:.4f}")
print(f"Precision: {precision_rf:.4f}")
print(f"Recall: {recall_rf:.4f}")
print(f"F1-score: {f1_rf:.4f}")

Random Forest Classifier Evaluation:
Accuracy: 0.7878
Precision: 0.7942
Recall: 0.7825
F1-score: 0.7883


- Gradient Boosting

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming X and y are your feature matrix and target vector
# and they have been split into X_train, X_test, y_train, y_test
# Tree-based models like Gradient Boosting generally work well with unscaled data

# Initialize the Gradient Boosting Classifier
# You can adjust hyperparameters like n_estimators, learning_rate, max_depth, etc.
gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Train the Gradient Boosting model
gb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_gb = gb_classifier.predict(X_test)

# Evaluate the Gradient Boosting model
accuracy_gb = accuracy_score(y_test, y_pred_gb)
precision_gb = precision_score(y_test, y_pred_gb)
recall_gb = recall_score(y_test, y_pred_gb)
f1_gb = f1_score(y_test, y_pred_gb)

# Print the evaluation metrics
print("Gradient Boosting Classifier Evaluation:")
print(f"Accuracy: {accuracy_gb:.4f}")
print(f"Precision: {precision_gb:.4f}")
print(f"Recall: {recall_gb:.4f}")
print(f"F1-score: {f1_gb:.4f}")

Gradient Boosting Classifier Evaluation:
Accuracy: 0.7832
Precision: 0.7543
Recall: 0.8462
F1-score: 0.7976


- Adaptive Boosting

In [15]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming X and y are your feature matrix and target vector
# and they have been split into X_train, X_test, y_train, y_test
# Tree-based models like AdaBoost generally work well with unscaled data

# Initialize the base estimator (typically a Decision Tree with max_depth=1 - a stump)
base_estimator = DecisionTreeClassifier(max_depth=1)

# Initialize the AdaBoost Classifier
# You can adjust hyperparameters like n_estimators, learning_rate, estimator, etc.
adaboost_classifier = AdaBoostClassifier(estimator=base_estimator, n_estimators=50, learning_rate=1.0, random_state=42)

# Train the AdaBoost model
adaboost_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_ab = adaboost_classifier.predict(X_test)

# Evaluate the AdaBoost model
accuracy_ab = accuracy_score(y_test, y_pred_ab)
precision_ab = precision_score(y_test, y_pred_ab)
recall_ab = recall_score(y_test, y_pred_ab)
f1_ab = f1_score(y_test, y_pred_ab)

# Print the evaluation metrics
print("AdaBoost Classifier Evaluation:")
print(f"Accuracy: {accuracy_ab:.4f}")
print(f"Precision: {precision_ab:.4f}")
print(f"Recall: {recall_ab:.4f}")
print(f"F1-score: {f1_ab:.4f}")

AdaBoost Classifier Evaluation:
Accuracy: 0.7688
Precision: 0.7717
Recall: 0.7699
F1-score: 0.7708


Which model is the best and why?

In [ ]:
# The Random Forest Classifier is clearly the best performing model among those you've evaluated so far.
# Here's why:
#
# Significantly Higher Accuracy: The Random Forest achieved an accuracy of approximately 78.8%,
# which is drastically higher than the accuracy of the Bagging (around 49.5%) and Pasting (around 49.5%)
# classifiers using KNN as the base estimator. This means the Random Forest correctly predicted the
# transportation status of a much larger proportion of the passengers in the test set.
#
# Balanced Precision and Recall: The Random Forest also shows a good balance between precision
# (avoiding false positives) and recall (avoiding false negatives), as indicated by the relatively
# close values of these metrics and a strong F1-score.
#
# Poor Performance of Bagging and Pasting with KNN: The Bagging and Pasting classifiers,
# using KNN with n_neighbors=5 as the base estimator, are performing at a level close to random
# guessing (an accuracy of 0.5 would be expected from a purely random binary classifier).
# This suggests that either:
#   - KNN with n_neighbors=5 is not a suitable base estimator for this problem, even within an ensemble.
#   - The number of estimators (set to 5 in your provided Bagging/Pasting code) is too low to provide
#     any significant benefit from the ensemble.
#   - There might be other issues with how KNN is interacting with the high-dimensional, sparse data
#     after one-hot encoding, even after scaling.
#
# In conclusion, based on the metrics you've shared, the Random Forest Classifier is the superior
# model due to its much higher accuracy and balanced precision and recall compared to the Bagging
# and Pasting ensembles using KNN.